# NAWI Alliance Spring Meeting 2023: 
# Interactive Code Demonstration Using WaterTAP

### Today's demonstration will show 
- Part 1: how to build, initialize, simulate, and optimize a nanofiltration (NF) unit model based on the Donnan Steric Pore Model with Dielectric Exclusion (DSPM-DE).
- Part 2: repeat the same process for an NF flowsheet with bypass.
- Part 3: demonstrate the same NF flowsheet in WaterTAP's graphical user interface, which is in early stages of development.

# Part 1: Build, setup, simulate, and optimize the NF DSPM-DE unit model

<img src="nf_dspmde_transport_mechanisms.png" width="500" height="340">
Image source: <a href="https://doi.org/10.1016/j.desal.2017.07.020">Roy et al., 2017</a>


## Step 1: Import libraries.

In [1]:
# Imports from Pyomo, including "value" for getting the 
# value of Pyomo objects
from pyomo.environ import ConcreteModel, Objective, Expression, value, units as pyunits
# Imports from IDAES
# Import flowsheet block from IDAES core
from idaes.core import FlowsheetBlock
# Import function to get default solver
from idaes.core.solvers import get_solver
# Import function to check degrees of freedom
from idaes.core.util.model_statistics import degrees_of_freedom
# Import utility function for calculating scaling factors
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor

# Imports from WaterTAP
# Import MultiComponent Aqueous Solution property model
from watertap.property_models.multicomp_aq_sol_prop_pack import (MCASParameterBlock, 
                                                                 ActivityCoefficientModel,
                                                                 DensityCalculation,)
# Import RO model
from watertap.unit_models.nanofiltration_DSPMDE_0D import (
    NanofiltrationDSPMDE0D,
    MassTransferCoefficient,
    ConcentrationPolarizationType,
)

## Step 2: Start building the NF DSPM-DE model.

In [2]:
# Create a Pyomo concrete model, flowsheet, and set up solute properties required by the DSPM-DE model.
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

property_kwds = {
"solute_list": [
    "Ca_2+",
    "SO4_2-",
    "HCO3_-",
    "Na_+",
    "Cl_-",
],
"diffusivity_data": {
    ("Liq", "Ca_2+"): 9.2e-10,
    ("Liq", "SO4_2-"): 1.06e-9,
    ("Liq", "HCO3_-"): 1.19e-9,
    ("Liq", "Na_+"): 1.33e-9,
    ("Liq", "Cl_-"): 2.03e-9,
},
"mw_data": {
    "H2O": 18e-3,
    "Ca_2+": 40e-3,
    "HCO3_-": 61.0168e-3,
    "SO4_2-": 96e-3,
    "Na_+": 23e-3,
    "Cl_-": 35e-3,
},
"stokes_radius_data": {
    "Ca_2+": 0.309e-9,
    "HCO3_-": 2.06e-10,
    "SO4_2-": 0.230e-9,
    "Cl_-": 0.121e-9,
    "Na_+": 0.184e-9,
},
"charge": {
    "Ca_2+": 2,
    "HCO3_-": -1,
    "SO4_2-": -2,
    "Na_+": 1,
    "Cl_-": -1,
},
"activity_coefficient_model": ActivityCoefficientModel.ideal,
"density_calculation": DensityCalculation.constant,
}

m.fs.properties = MCASParameterBlock(**property_kwds)

In [3]:
# Add an NF DSPM-DE unit to the flowsheet.
m.fs.unit = NanofiltrationDSPMDE0D(property_package=m.fs.properties)

## Step 3: Specify values for system variables.

In [4]:
mass_flow_in = 1 * pyunits.kg / pyunits.s
feed_mass_frac = {
    "Ca_2+": 4.0034374454637006e-04,
    "HCO3_-": 0.00022696833343821863,
    "SO4_2-": 0.00020497140244420624,
    "Cl_-": 0.0004559124032433401,
    "Na_+": 0.00043333830389924205,
}

# Fix mole flow rates of each ion and water
for ion, x in feed_mass_frac.items():
    mol_comp_flow = (
        x
        * pyunits.kg
        / pyunits.kg
        * mass_flow_in
        / m.fs.unit.feed_side.properties_in[0].mw_comp[ion]
    )
    m.fs.unit.inlet.flow_mol_phase_comp[0, "Liq", ion].fix(mol_comp_flow)
H2O_mass_frac = 1 - sum(x for x in feed_mass_frac.values())
H2O_mol_comp_flow = (
    H2O_mass_frac
    * pyunits.kg
    / pyunits.kg
    * mass_flow_in
    / m.fs.unit.feed_side.properties_in[0].mw_comp["H2O"]
)
m.fs.unit.inlet.flow_mol_phase_comp[0, "Liq", "H2O"].fix(H2O_mol_comp_flow)

# Use assert electroneutrality method from property model to ensure the ion concentrations provided
# obey electroneutrality condition
m.fs.unit.feed_side.properties_in[0].assert_electroneutrality(
    defined_state=True,
    adjust_by_ion="Cl_-",
    get_property="mass_frac_phase_comp",
)

# Fix other inlet state variables
m.fs.unit.inlet.temperature[0].fix(298.15)
m.fs.unit.inlet.pressure[0].fix(4e5)

# Fix the membrane variables that are usually fixed for the DSPM-DE model
m.fs.unit.radius_pore.fix(0.5e-9)
m.fs.unit.membrane_thickness_effective.fix(1.33e-6)
m.fs.unit.membrane_charge_density.fix(-27)
m.fs.unit.dielectric_constant_pore.fix(41.3)

# Fix final permeate pressure to be ~atmospheric
m.fs.unit.mixed_permeate[0].pressure.fix(101325)

m.fs.unit.spacer_porosity.fix(0.85)
m.fs.unit.channel_height.fix(5e-4)
m.fs.unit.velocity[0, 0].fix(0.25)
m.fs.unit.area.fix(50)

# Fix additional variables for calculating mass transfer coefficient with spiral wound correlation
m.fs.unit.spacer_mixing_efficiency.fix()
m.fs.unit.spacer_mixing_length.fix()

{Member of conc_mol_phase_comp} : Molar concentration
    Size=6, Index=fs.unit.feed_side.properties_in[0.0].conc_mol_phase_comp_index, Units=mol/m**3
    Key            : Lower : Value             : Upper : Fixed : Stale : Domain
    ('Liq', 'H2O') :     0 : 55425.30350471729 :  None : False : False :  Reals
{Member of conc_mol_phase_comp} : Molar concentration
    Size=6, Index=fs.unit.feed_side.properties_in[0.0].conc_mol_phase_comp_index, Units=mol/m**3
    Key              : Lower : Value              : Upper : Fixed : Stale : Domain
    ('Liq', 'Ca_2+') :     0 : 10.002347479606016 :  None : False : False :  Reals
{Member of conc_mol_phase_comp} : Molar concentration
    Size=6, Index=fs.unit.feed_side.properties_in[0.0].conc_mol_phase_comp_index, Units=mol/m**3
    Key               : Lower : Value              : Upper : Fixed : Stale : Domain
    ('Liq', 'SO4_2-') :     0 : 2.1337862967321106 :  None : False : False :  Reals
{Member of conc_mol_phase_comp} : Molar concentration

## Step 4: Scale all variables.

In [9]:
# Calculate scaling factors for variables.
calculate_scaling_factors(m);


2023-05-17 18:44:53 [WARNING] idaes.watertap.core.util.scaling: If there was a property constraint written for the variable, fs.properties.diffus_phase_comp, that constraint was not scaled. The transform_property_constraints tool expects constraints to have the following naming convention: 'eq_' + 'diffus_phase_comp'. This suggests that the user may have defined a property in metadata but failed to follow the naming convention for its constraint. If there is no property constraint associated with the diffus_phase_comp, this warning can be ignored.
2023-05-17 18:44:53 [WARNING] idaes.watertap.core.util.scaling: If there was a property constraint written for the variable, fs.properties.diffus_phase_comp, that constraint was not scaled. The transform_property_constraints tool expects constraints to have the following naming convention: 'eq_' + 'diffus_phase_comp'. This suggests that the user may have defined a property in metadata but failed to follow the naming convention for its constra

## Step 5: Initialize the model.

In [10]:
m.fs.unit.initialize()

2023-05-17 18:46:42 [INFO] idaes.init.fs.unit: Initialization Complete: optimal - Optimal Solution Found


## Step 6: Setup a solver and run a simulation.

In [11]:
# Check that degrees of freedom = 0 before attempting simulation.
# This means that the performance of the flowsheet is completely
# determined by the system variables that were fixed above.
assert degrees_of_freedom(m) == 0

In [12]:
# Setup solver
solver = get_solver()

In [13]:
# Run simulation
simulation_results = solver.solve(m)

In [15]:
# Display report, reports include a small subset of the most important variables
m.fs.unit.report()


Unit : fs.unit                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                                                           : Value      : Units                                        : Fixed : Bounds
                             Electric Potential @ Permeate, Inlet :   0.016760 : kilogram * meter ** 2 / ampere / second ** 3 : False : (None, None)
                            Electric Potential @ Permeate, Outlet :   0.016097 : kilogram * meter ** 2 / ampere / second ** 3 : False : (None, None)
                        Electric Potential @ Pore Entrance, Inlet :  -0.061764 : kilogram * meter ** 2 / ampere / second ** 3 : False : (None, None)
                       Electric Potential @ Pore Entrance, Outlet :  -0.060043 : kilogram * meter ** 2 / ampere / second ** 3 : False : (None, None)
                            Electric Potential @ Po

    Key                                                         : Value      : Units
                                    Average Mole FLux of Ca_2+  : 2.0037e-05 : mole / meter ** 2 / second
                                     Average Mole FLux of Cl_-  : 8.0278e-05 : mole / meter ** 2 / second
                                      Average Mole FLux of H2O  :    0.20170 : mole / meter ** 2 / second
                                   Average Mole FLux of HCO3_-  : 8.7832e-06 : mole / meter ** 2 / second
                                     Average Mole FLux of Na_+  : 4.9013e-05 : mole / meter ** 2 / second
                                   Average Mole FLux of SO4_2-  : 1.2736e-08 : mole / meter ** 2 / second
                                  Average Volumetric Flux (LMH) :     13.111 : meter / second
             Born Solvation Energy Partitioning Factor of Ca_2+ :   0.013960 : dimensionless
              Born Solvation Energy Partitioning Factor of Cl_- :   0.065409 : dimensionless

In [16]:
# Display all results, this shows all variables and constraints
m.fs.unit.display()

Block fs.unit

  Variables:
    _flow_mol_phase_comp_inlet_ref : Size=6, Index=fs.unit._flow_mol_phase_comp_inlet_ref_index, ReferenceTo=fs.unit.feed_side.properties_in[...].component('flow_mol_phase_comp')[...]
        Key                    : Lower : Value                 : Upper : Fixed : Stale : Domain
         (0.0, 'Liq', 'Ca_2+') :     0 :  0.010008593613659252 :  None :  True :  True : NonNegativeReals
          (0.0, 'Liq', 'Cl_-') :     0 :   0.03086797760732251 :  None :  True :  True : NonNegativeReals
           (0.0, 'Liq', 'H2O') :     0 :     55.45991476735715 :  None :  True :  True : NonNegativeReals
        (0.0, 'Liq', 'HCO3_-') :     0 :  0.003719767890781205 :  None :  True :  True : NonNegativeReals
          (0.0, 'Liq', 'Na_+') :     0 :  0.018840795821706176 :  None :  True :  True : NonNegativeReals
        (0.0, 'Liq', 'SO4_2-') :     0 : 0.0021351187754604815 :  None :  True :  True : NonNegativeReals
    _temperature_inlet_ref : Size=1, Index=fs._time, Ref

## Step 7: Unfix variables, set variable bounds, and run optimization to minimize specific energy consumption.

In [17]:
# Unfix membrane area and feed pressure
m.fs.unit.area.unfix()                  # membrane area (m^2)
m.fs.unit.inlet.pressure[0].unfix()     # feed pressure (Pa)

In [18]:
# Set lower and upper bounds for membrane area (m^2)
m.fs.unit.area.setlb(1)
m.fs.unit.area.setub(500)

In [19]:
# Set lower and upper bounds for feed pressure (Pa)
m.fs.unit.inlet.pressure[0].setlb(10e5)
m.fs.unit.inlet.pressure[0].setub(80e5)

In [20]:
# Assume 100% efficiency of pumps and ERD and no pressure losses
#--> Pump power consumption ~ Qp*Pf/3.6e6
m.fs.specific_energy_consumption = Expression(
    expr=m.fs.unit.inlet.pressure[0]/(3.6e6))

In [21]:
# Define objective function to minimize the specific energy consumption.
m.fs.objective = Objective(expr=m.fs.specific_energy_consumption)

In [22]:
# Set the water recovery to 50%
m.fs.unit.recovery_vol_phase[0,'Liq'].fix(0.50)

In [23]:
# The solver will find the membrane area and 
# inlet pressure that achieve 50% recovery while minimizing
# specific energy consumption. Since we fixed the 
# volumetric water recovery, a degree of freedom 
# was removed from the model and is now 1.
print(degrees_of_freedom(m))

1


In [24]:
# Solve the model
optimization_results = solver.solve(m)
print(optimization_results)


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 470
  Number of variables: 471
  Sense: unknown
Solver: 
- Status: ok
  Message: Ipopt 3.13.2\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 1.7112939357757568
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [25]:
# membrane area of the optimized RO unit
value(m.fs.unit.area)

106.42309519006501

In [26]:
# inlet pressure of the optimized RO unit
value(m.fs.unit.inlet.pressure[0])

1000000.0089212639

In [27]:
# the minimum specific energy consumption
value(m.fs.specific_energy_consumption)

0.2777777802559066

In [28]:
# display the overall report on the RO unit
m.fs.unit.report()


Unit : fs.unit                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance

    Variables: 

    Key                                                           : Value      : Units                                        : Fixed : Bounds
                             Electric Potential @ Permeate, Inlet :   0.031602 : kilogram * meter ** 2 / ampere / second ** 3 : False : (None, None)
                            Electric Potential @ Permeate, Outlet :   0.031594 : kilogram * meter ** 2 / ampere / second ** 3 : False : (None, None)
                        Electric Potential @ Pore Entrance, Inlet :  -0.042186 : kilogram * meter ** 2 / ampere / second ** 3 : False : (None, None)
                       Electric Potential @ Pore Entrance, Outlet :  -0.038950 : kilogram * meter ** 2 / ampere / second ** 3 : False : (None, None)
                            Electric Potential @ Po

    Key                                                         : Value      : Units
                                    Average Mole FLux of Ca_2+  : 4.8357e-05 : mole / meter ** 2 / second
                                     Average Mole FLux of Cl_-  : 0.00014295 : mole / meter ** 2 / second
                                      Average Mole FLux of H2O  :    0.26064 : mole / meter ** 2 / second
                                   Average Mole FLux of HCO3_-  : 1.6889e-05 : mole / meter ** 2 / second
                                     Average Mole FLux of Na_+  : 6.5057e-05 : mole / meter ** 2 / second
                                   Average Mole FLux of SO4_2-  : 9.6856e-07 : mole / meter ** 2 / second
                                  Average Volumetric Flux (LMH) :     16.942 : meter / second
             Born Solvation Energy Partitioning Factor of Ca_2+ :   0.013960 : dimensionless
              Born Solvation Energy Partitioning Factor of Cl_- :   0.065409 : dimensionless